# RSVD


In [1]:
import numpy as np
import jax
import jax.numpy as jnp

In [2]:
def truncated_svd(A, k):
    """
    Compute the truncated SVD of a matrix A using NumPy.

    Parameters:
        A (ndarray): Input matrix of shape (m, n).
        k (int): Number of singular values and vectors to retain.

    Returns:
        U_k (ndarray): Truncated left singular vectors, shape (m, k).
        S_k (ndarray): Truncated singular values, shape (k,).
        Vt_k (ndarray): Truncated right singular vectors, shape (k, n).
    """
    # Perform full SVD
    U, S, Vt = np.linalg.svd(A, full_matrices=False)

    # Truncate to the first k singular values/vectors
    U_k = U[:, :k]
    S_k = S[:k]
    Vt_k = Vt[:k, :]

    return U_k, S_k, Vt_k

## SVR


In [ ]:
class SVR:
    def __init__(self, epsilon=0.1, lmbda=1.0):
        self.epsilon = epsilon
        self.lmbda = lmbda
        self.w = None

    def loss(self, params, X, y):
        predictions = jnp.dot(X, params[:-1]) + params[-1]

        # Compute epsilon-insensitive loss
        epsilon_loss = jnp.maximum(0, jnp.abs(predictions - y) - self.epsilon)

        # Regularization term (L2 norm of w)
        reg_term = self.lmbda * jnp.sum(params**2)

        # Total loss
        return reg_term + jnp.mean(epsilon_loss)

    def train(self, X, y):
        _, n_features = X.shape

        # Initialize weights and bias
        self.w = jnp.zeros(n_features + 1)

        # Solve optimization problem
        opt_res = jax.scipy.optimize.minimize(
            self.loss, self.w, method="BFGS", args=(X, y)
        )
        self.w = opt_res.x

    def predict(self, X):
        return jnp.dot(X, self.w[:-1]) + self.w[-1]

## SVM


In [ ]:
class SVM:
    def __init__(self, lmbda=1.0):
        self.lmbda = lmbda
        self.w = None

    def loss(self, params, X, y):
        # Compute the decision function
        decision = jnp.dot(X, params[:-1]) + params[-1]
        # Compute the hinge loss
        loss_val = jnp.maximum(0, 1 - y * decision)
        # Regularization term (L2 norm of w)
        reg_term = self.lmbda * jnp.sum(params**2)
        # Total loss
        return reg_term + jnp.mean(loss_val)

    def train(self, X, y):
        _, n_features = X.shape

        # Initialize weights and bias
        self.w = jnp.zeros(n_features + 1)

        # Solve optimization problem
        opt_res = jax.scipy.optimize.minimize(
            self.loss, self.w, method="BFGS", args=(X, y)
        )
        self.w = opt_res.x

    def predict(self, X):
        # Decision function
        decision = jnp.dot(X, self.w[:-1]) + self.w[-1]
        return jnp.sign(decision)

## Gradient Descent


In [ ]:
def gradient_descent(func, dfunc, x_0, stepsize, tol, max_iter):
    """
    Perform gradient descent to minimize a given function.

    Parameters:
    func (callable): The function to minimize
    dfunc (callable): The gradient of the function
    x_0 (np.array): Initial guess for the minimizer
    stepsize (float): Constant step size for gradient descent
    tol (float): Tolerance for stopping criterion
    max_iter (int): Maximum number of iterations

    Returns:
    np.array: The point minimizing the function
    float: The minimum value of the function
    list: List of points visited during gradient descent
    """
    x = np.array(x_0, dtype=float)  # Initialize current point
    path = [x.copy()]  # Store the path of points visited

    for iteration in range(max_iter):
        grad = dfunc(x)  # Compute gradient
        x_new = x - stepsize * grad  # Update step using gradient descent

        # Save the new point in the path
        path.append(x_new.copy())

        # Check for convergence
        if np.linalg.norm(x_new - x) < tol:
            print(f"Converged in {iteration + 1} iterations.")
            return x_new, func(x_new), path

        x = x_new  # Update current point

    print("Reached the maximum number of iterations.")
    return x, func(x), path

## Stochastic Gradient Descent

In [3]:
from sympy import symbols, diff

x, y = symbols("x y")
f = x**2 + 3 * x * y + y**2
grad_f = (diff(f, x), diff(f, y))
print(grad_f)

(2*x + 3*y, 3*x + 2*y)


Compute Hessian Matrix


In [4]:
from sympy import symbols, diff, Matrix

# Define the variables
x, y = symbols("x y")

# Define the function
f = 100 * (y - x**2) ** 2 + (1 - x) ** 2

# Compute the first-order partial derivatives
f_x = diff(f, x)
f_y = diff(f, y)

# Compute the second-order partial derivatives
f_xx = diff(f_x, x)
f_xy = diff(f_x, y)
f_yy = diff(f_y, y)

# Create the Hessian matrix
hessian = Matrix([[f_xx, f_xy], [f_xy, f_yy]])

# Display the Hessian matrix
print("Hessian Matrix:")
print(hessian)


Hessian Matrix:
Matrix([[1200*x**2 - 400*y + 2, -400*x], [-400*x, 200]])


In [5]:
import numpy as np
from sympy import symbols, diff, hessian, solve

# Define symbols
x, y = symbols("x y")

# Define the function
f = 100 * (y - x**2) ** 2 + (1 - x) ** 2

# Compute the gradient
grad_f = [diff(f, var) for var in (x, y)]

# Compute the Hessian
hessian_f = hessian(f, (x, y))

# Solve for critical points (gradient = 0)
critical_points = solve(grad_f, (x, y))

# Print results
print("Gradient:", grad_f)
print("Hessian:\n", hessian_f)
print("Critical Points:", critical_points)


Gradient: [-400*x*(-x**2 + y) + 2*x - 2, -200*x**2 + 200*y]
Hessian:
 Matrix([[1200*x**2 - 400*y + 2, -400*x], [-400*x, 200]])
Critical Points: [(1, 1)]
